# Part 4: Interactive Guide for Agentic RAG with MCP (Local Setup)

**[EN]** This notebook is an interactive guide for demonstrating an Agentic RAG workflow on your local machine. It will guide you through setting up the Elastic MCP Server and using an external AI agent (like Claude Desktop) to perform advanced searches on Elasticsearch.<br>
**[KR]** 이 노트북은 당신의 로컬 컴퓨터에서 Agentic RAG 워크플로우를 시연하기 위한 대화형 가이드입니다. Elastic MCP 서버를 설정하고 Claude Desktop과 같은 외부 AI 에이전트를 사용하여 Elasticsearch에 대한 고급 검색을 수행하는 과정을 안내합니다.

## Part 1: Prerequisites

**[EN]** Before you begin, please ensure you have the following installed on your local machine:<br>
**[KR]** 시작하기 전에, 당신의 로컬 컴퓨터에 다음 프로그램들이 설치되어 있는지 확인해주세요:

1.  **Python**: This project is based on Python. Ensure you have a working Python environment and have installed the dependencies from `requirements.txt`.
2.  **Node.js**: The MCP Server is a Node.js application. Please install the latest LTS version from [nodejs.org](https://nodejs.org/).
3.  **Claude Desktop**: This demo uses Claude Desktop as the AI agent. Make sure it is installed on your machine.

## Part 2: Server Execution and Connection (Manual Guide)

**[EN]** The following steps require manual execution in your local terminal.<br>
**[KR]** 다음 단계들은 당신의 로컬 터미널에서 직접 실행해야 합니다.

### Step 2.1: Create `package.json` and Install MCP Server

**[EN]** This cell creates a `package.json` file. After running it, you will need to run `npm install` in your terminal.<br>
**[KR]** 이 셀은 `package.json` 파일을 생성합니다. 셀을 실행한 후, 터미널에서 `npm install`을 실행해야 합니다.

In [ ]:
import json

package_json = {
    "name": "colpali-mcp-demo",
    "version": "1.0.0",
    "description": "A demo project to connect Claude Desktop with Elasticsearch via MCP.",
    "dependencies": {
        "@elastic/mcp-server-elasticsearch": "latest"
    }
}

with open('package.json', 'w') as f:
    json.dump(package_json, f, indent=2)

print("package.json created successfully.")
print("\nNext, open your terminal in this project directory and run: npm install")

### Step 2.2: Run the MCP Server in Your Local Terminal

1. **Open a new terminal** (e.g., Command Prompt, PowerShell, Terminal.app) in this project's root directory.
2. **Copy and paste the commands below** for your operating system. It will load your Elasticsearch credentials and start the server.

**For macOS/Linux:**
```bash
# (You can copy the credentials from the output of the next cell)
export ES_URL="<your-elasticsearch-url>"
export ES_API_KEY="<your-api-key>"
npx @elastic/mcp-server-elasticsearch
```
**For Windows (Command Prompt):**
```cmd
set ES_URL="<your-elasticsearch-url>"
set ES_API_KEY="<your-api-key>"
npx @elastic/mcp-server-elasticsearch
```
**Important**: Keep this terminal running. The server must be active for the AI agent to connect.

In [ ]:
from dotenv import load_dotenv
import os

dotenv_path = 'elastic.env'
load_dotenv(dotenv_path=dotenv_path)
ES_URL = os.getenv("ES_URL", "")
ES_API_KEY = os.getenv("ES_API_KEY", "")

print("--- Copy these values for the terminal command above ---")
print(f"Your ES_URL is: {ES_URL}")
print(f"Your ES_API_KEY is: {ES_API_KEY}")

### Step 2.3: Configure Claude Desktop

1. Open the **Claude Desktop App**.
2. Navigate to `Settings > Developer > MCP Servers` and click **'Edit Config'**.
3. Add the following JSON configuration. Since the server is running on your local machine, you can connect to it directly via `localhost`.

```json
{
    "mcpServers": {
        "elasticsearch-mcp-server": {
            "url": "http://localhost:8889/mcp/v1"
        }
    }
}
```
4. Save the file. Claude should now be connected to your local MCP server.

## Part 3: Agentic RAG Demonstration (Interactive)

**[EN]** Now, we will simulate a customer's request. This cell will generate a complete, advanced Elasticsearch query based on your input. You will then copy this query and run it in Claude Desktop to demonstrate the Agentic RAG workflow.<br>
**[KR]** 이제 고객의 요청을 시뮬레이션합니다. 이 셀은 사용자의 입력에 따라 완전하고 진보된 Elasticsearch 쿼리를 생성합니다. 이 쿼리를 복사하여 Claude Desktop에서 실행함으로써 Agentic RAG 워크플로우를 시연합니다.

In [ ]:
import ipywidgets as widgets
from IPython.display import display, Markdown
import json
from colpali_engine.models import ColQwen2_5, ColQwen2_5_Processor
import torch
import numpy as np

# --- Helper functions (already defined in other notebooks, included here for completeness) ---
device_map = "cpu"
if torch.backends.mps.is_available(): device_map = "mps"
elif torch.cuda.is_available(): device_map = "cuda:0"
MODEL_NAME = "tsystems/colqwen2.5-3b-multilingual-v1.0"
model = ColQwen2_5.from_pretrained(MODEL_NAME, torch_dtype=torch.bfloat16 if device_map != "cpu" else torch.float32, device_map=device_map).eval()
processor = ColQwen2_5_Processor.from_pretrained(MODEL_NAME)

def create_colqwen_query_vectors(query_text, model, processor):
    inputs = processor.process_queries([query_text]).to(model.device)
    with torch.no_grad(): outputs = model(**inputs)
    return outputs.cpu().to(torch.float32).numpy().tolist()[0]

def to_avg_vector(vectors):
    vectors_array = np.array(vectors)
    avg_vector = np.mean(vectors_array, axis=0)
    norm = np.linalg.norm(avg_vector)
    return (avg_vector / norm).tolist() if norm > 0 else avg_vector.tolist()

# --- Interactive UI --- 
query_input = widgets.Text(value='Do you have a benefits policy change notice from HR?', description='Query:', layout=widgets.Layout(width='95%'))
mode_selector = widgets.RadioButtons(options=['A. Full Colpali Search', 'B. KNN Search Only', 'C. KNN + Rescore'], description='Search Mode:')
generate_button = widgets.Button(description="Generate Agent Command")
output_area = widgets.Output()

def generate_agent_command(b):
    output_area.clear_output()
    query_text = query_input.value
    search_mode = mode_selector.value
    
    with output_area:
        print("Generating vectors and Elasticsearch query...")
        query_multi_vectors = create_colqwen_query_vectors(query_text, model, processor)
        es_query_body = {}
        index_name = ""

        if search_mode.startswith('A.'):
            index_name = "colqwen-rvlcdip-demo-part1"
            es_query_body = {"size": 5, "query": {"script_score": {"query": {"match_all": {}}, "script": {"source": "maxSimDotProduct(params.query_vector, 'colqwen_vectors')", "params": {"query_vector": query_multi_vectors}}}}}
        else:
            index_name = "colqwen-rvlcdip-demo-part2"
            query_avg_vector = to_avg_vector(query_multi_vectors)
            knn_query = {"field": "colqwen_avg_vector", "query_vector": query_avg_vector, "k": 200, "num_candidates": 500}
            if search_mode.startswith('B.'):
                es_query_body = {"size": 5, "knn": knn_query}
            else: # C. KNN + Rescore
                rescore_def = {"window_size": 50, "query": {"rescore_query": {"script_score": {"query": {"match_all": {}}, "script": {"source": "maxSimDotProduct(params.query_vector, 'colqwen_vectors')", "params": {"query_vector": query_multi_vectors}}}}, "query_weight": 0.0, "rescore_query_weight": 1.0}}
                es_query_body = {"size": 5, "knn": knn_query, "rescore": rescore_def}
        
        # Pretty print the JSON query
        query_json_string = json.dumps(es_query_body, indent=2)
        
        # Generate the final prompt for the AI Agent
        agent_prompt = f"""@elasticsearch-mcp-server search index {index_name} with query: {query_json_string}"""
        
        print("Done!\n")
        display(Markdown("**Copy the following command and paste it into Claude Desktop:**"))
        display(Markdown(f"```\n{agent_prompt}\n```"))

generate_button.on_click(generate_agent_command)
display(widgets.VBox([query_input, mode_selector, generate_button, output_area]))